# Initialize Examples and Experiment Functions

In [57]:
%reload_ext autoreload
%autoreload 2

from boat_models import Boat, twoInputBoat, threeInputBoat
import numpy as np
import pickle
from IPython.display import HTML
from states_initialization import states_initialization
from trajectory_planner import BoatConfigurationPlanning, bold

def anim_plot(split, i):
    print '%f seconds' % time_configurations[split][i-1][-1]
    return HTML(anim_configurations[split][i-1])

def to_x0_xN(xy0, xyN, final_offset):
    num_boats = len(xy0)
    
    x0 = np.zeros((len(xy0), Boat.num_states))
    xN = np.zeros((len(xy0), Boat.num_states))

    x0[:,:2]=xy0
    xN[:,:2]=xyN
    xN[:,:2]+=final_offset
    
    return (x0,xN)      

def experiment(boat, x0, xN, steps=100, dt_init=0.4, expansion_N=None, shapemove_N=None, min_time=5, max_time=100, expansion_factor=1.4, initialize=True, example_name=None, init_name=None):
    boats_S_init, assignments = states_initialization(x0, xN, steps, dt_init, expansion_factor=expansion_factor, expansion_N=expansion_N, shapemove_N=shapemove_N)
    return boats_S_init, assignments
    xN_assigned = xN[assignments]
    planner = BoatConfigurationPlanning(boat)
    initialization = boats_S_init if initialize else None
    S, U, time, mp, result, solve_time = planner.compute_trajectory(min_time, max_time, x0, xN_assigned, step_initialization=dt_init, states_initialization=initialization)
    return {
            'S_init':boats_S_init, 'S':S, 'U':U, 'x0':x0, 'xN':xN, 'N':steps, 'time':time, 'result':str(result), 'solve_time':solve_time, 'example_name': example_name, 'init_name':init_name, 'boat_type': boat.__class__.__name__,
            'params':{'expansion_N':expansion_N, 'shapemove_N':shapemove_N, 'expansion_factor':expansion_factor, 'min_time':min_time, 'max_time':max_time, 'dt_init':dt_init}
           }

def run_experiments(boats, examples):
    experiments = {}  
    for boat in boats:
        print bold('====%s EXPERIMENTS====\n' % boat.__class__.__name__)
        boat_experiments = {}
        #print bold('----NO_INIT----\n')
        #boat_experiments['no_init']              = [experiment(boat,x0,xN,initialize=False, example_name=name, init_name='none')                                               for x0,xN,name in examples]
        print bold('----INTERPOLATE----')
        boat_experiments['init']                 = [experiment(boat,x0,xN, example_name=name, init_name='interpolate')                                                         for x0,xN,name in examples]
        print bold('----INTERPOLATE EXPANSION----')
        boat_experiments['init_expansion']       = [experiment(boat,x0,xN, expansion_N=20, example_name=name, init_name='interpolate expansion')                               for x0,xN,name in examples]
        print bold('----INTERPOLATE SHAPE----')
        boat_experiments['init_shape']           = [experiment(boat,x0,xN,shapemove_N=50, example_name=name, init_name='interpolate shape')                                    for x0,xN,name in examples]
        print bold('----INTERPOLATE SHAPE EXPANSION----')
        boat_experiments['init_shape_expansion'] = [experiment(boat,x0,xN,shapemove_N=50, expansion_N=20, example_name=name, init_name='interpolate expansion and shape') for x0,xN,name in examples]
        experiments[boat.__class__.__name__] = boat_experiments
    return experiments

In [2]:
examples = (
        #1
        (((0, 0),),
         ((0, 0),)),

        #2 rotate
        (((0, 0), (0, 1.5)),
         ((1, 0), (2.5, 0))),

        #4 line rotate
        (((2.25, 0), (0.75, 0), (-.75, 0), (-2.25, 0)),
         ((0, 2.25), (0, 0.75), (0, -.75), (0, -2.25))),

        #4 line to 4 grid
        (((2.25, 0), (0.75, 0), (-.75, 0), (-2.25, 0)),
         ((0, 0), (0, 1.5), (1.5, 0), (1.5, 1.5))),

        #4 grid rotate
        (((0, 0), (0, 1.5), (1.5, 0), (1.5, 1.5)),
         ((0, 1.06), (0, -1.06), (-1.06, 0), (1.06, 0))),

        #4 triangle rotate
        (((0, 0), (0, 1.5), (-1.5, 0), (1.5, 0)),
         ((0, 0), (0, 1.5), (0, -1.5), (1.5, 0))),

        #9 triangle to 9 grid
        (((-1.5, -3), (-1.5, -1.5), (-1.5, 0), (-1.5, 1.5), (-1.5, 3), (0, -1.5), (0, 0), (0, 1.5), (1.5, 0)),
         ((-1.5, 1.5), (0, 1.5), (1.5, 1.5), (-1.5, 0), (0, 0), (1.5, 0), (-1.5, -1.5), (0, -1.5), (1.5, -1.5))),

        #9 line to 9 grid
        (((6,0), (4.5, 0), (3, 0), (1.5, 0), (0, 0), (-1.5, 0), (-3, 0), (-4.5, 0), (-6, 0)),
         ((-1.5, 1.5), (0, 1.5), (1.5, 1.5), (-1.5, 0), (0, 0), (1.5, 0), (-1.5, -1.5), (0, -1.5), (1.5, -1.5))),

        #9 grid rotate
        (((-1.5, 1.5), (0, 1.5), (1.5, 1.5), (-1.5, 0), (0, 0), (1.5, 0), (-1.5, -1.5), (0, -1.5), (1.5, -1.5)),
         ((0, 0), (2.12, 0), (-2.12, 0), (1.06, 1.06), (1.06, -1.06), (-1.06, 1.06), (-1.06, -1.06), (0, 2.12), (0, -2.12)))

    )

final_offset = ((4, 0))

example_states = tuple([to_x0_xN(xy0, xyN, final_offset) for (xy0, xyN) in examples])
example_names = ['1 Boat Move', '2 Boat Line Rotate','4 Boat Line Rotate', '4 Boat Line to Grid', '4 Boat Grid Rotate', '4 Boat Triangle Rotate', '9 Boat Triangle to Grid', '9 Boat Line to Grid', '9 Boat Grid Rotate']
examples = [(x0,xN,name) for (x0,xN),name in zip(example_states,example_names)]

assert len(example_names) == len(example_states)

# Run Experiments

In [20]:
boats = [twoInputBoat(), threeInputBoat(), Boat()]
experiments1 = run_experiments(boats, examples)

====twoInputBoat EXPERIMENTS====

----INTERPOLATE----
INITIALIZED 1 twoInputBoat boats
boats_S:(1, 101, 4), boats_U:(1, 100, 2), time_array:(101,)
Number of decision vars 605
0.008179 seconds
ADDING COSTS
0.000159 seconds
ADDING CONSTRAINTS
0.060986 seconds
PLANNING
SolutionResult.kSolutionFound
0.049717 seconds

INITIALIZED 2 twoInputBoat boats
boats_S:(2, 101, 4), boats_U:(2, 100, 2), time_array:(101,)
Number of decision vars 1209
0.004087 seconds
ADDING COSTS
0.000066 seconds
ADDING CONSTRAINTS
0.114375 seconds
PLANNING
SolutionResult.kSolutionFound
1.127982 seconds

INITIALIZED 4 twoInputBoat boats
boats_S:(4, 101, 4), boats_U:(4, 100, 2), time_array:(101,)
Number of decision vars 2417
0.008234 seconds
ADDING COSTS
0.000096 seconds
ADDING CONSTRAINTS
0.381859 seconds
PLANNING
SolutionResult.kSolutionFound
1.995942 seconds

INITIALIZED 4 twoInputBoat boats
boats_S:(4, 101, 4), boats_U:(4, 100, 2), time_array:(101,)
Number of decision vars 2417
0.020702 seconds
ADDING COSTS
0.000512 

SolutionResult.kSolutionFound
0.477990 seconds

INITIALIZED 4 twoInputBoat boats
boats_S:(4, 101, 4), boats_U:(4, 100, 2), time_array:(101,)
Number of decision vars 2417
0.008252 seconds
ADDING COSTS
0.000072 seconds
ADDING CONSTRAINTS
0.316734 seconds
PLANNING
SolutionResult.kSolutionFound
2.463798 seconds

INITIALIZED 4 twoInputBoat boats
boats_S:(4, 101, 4), boats_U:(4, 100, 2), time_array:(101,)
Number of decision vars 2417
0.008207 seconds
ADDING COSTS
0.000100 seconds
ADDING CONSTRAINTS
0.316453 seconds
PLANNING
SolutionResult.kSolutionFound
3.231044 seconds

INITIALIZED 4 twoInputBoat boats
boats_S:(4, 101, 4), boats_U:(4, 100, 2), time_array:(101,)
Number of decision vars 2417
0.008228 seconds
ADDING COSTS
0.000073 seconds
ADDING CONSTRAINTS
0.314779 seconds
PLANNING
SolutionResult.kSolutionFound
8.652233 seconds

INITIALIZED 4 twoInputBoat boats
boats_S:(4, 101, 4), boats_U:(4, 100, 2), time_array:(101,)
Number of decision vars 2417
0.007915 seconds
ADDING COSTS
0.000057 secon

SolutionResult.kSolutionFound
5.875794 seconds

INITIALIZED 4 threeInputBoat boats
boats_S:(4, 101, 6), boats_U:(4, 100, 3), time_array:(101,)
Number of decision vars 3625
0.012695 seconds
ADDING COSTS
0.000091 seconds
ADDING CONSTRAINTS
0.399078 seconds
PLANNING
SolutionResult.kSolutionFound
3.049655 seconds

INITIALIZED 4 threeInputBoat boats
boats_S:(4, 101, 6), boats_U:(4, 100, 3), time_array:(101,)
Number of decision vars 3625
0.011758 seconds
ADDING COSTS
0.000067 seconds
ADDING CONSTRAINTS
0.404227 seconds
PLANNING
SolutionResult.kSolutionFound
24.974589 seconds

INITIALIZED 4 threeInputBoat boats
boats_S:(4, 101, 6), boats_U:(4, 100, 3), time_array:(101,)
Number of decision vars 3625
0.011716 seconds
ADDING COSTS
0.000080 seconds
ADDING CONSTRAINTS
0.402068 seconds
PLANNING
SolutionResult.kSolutionFound
12.089206 seconds

INITIALIZED 9 threeInputBoat boats
boats_S:(9, 101, 6), boats_U:(9, 100, 3), time_array:(101,)
Number of decision vars 8155
0.030314 seconds
ADDING COSTS
0.00

0.577405 seconds
PLANNING
SolutionResult.kSolutionFound
19.980098 seconds

INITIALIZED 4 Boat boats
boats_S:(4, 101, 6), boats_U:(4, 100, 4), time_array:(101,)
Number of decision vars 4025
0.014048 seconds
ADDING COSTS
0.000101 seconds
ADDING CONSTRAINTS
0.578529 seconds
PLANNING
SolutionResult.kSolutionFound
20.761286 seconds

INITIALIZED 4 Boat boats
boats_S:(4, 101, 6), boats_U:(4, 100, 4), time_array:(101,)
Number of decision vars 4025
0.020286 seconds
ADDING COSTS
0.000159 seconds
ADDING CONSTRAINTS
0.575738 seconds
PLANNING
SolutionResult.kSolutionFound
18.896329 seconds

INITIALIZED 9 Boat boats
boats_S:(9, 101, 6), boats_U:(9, 100, 4), time_array:(101,)
Number of decision vars 9055
0.036351 seconds
ADDING COSTS
0.000100 seconds
ADDING CONSTRAINTS
1.744002 seconds
PLANNING
SolutionResult.kIterationLimit
647.323130 seconds

INITIALIZED 9 Boat boats
boats_S:(9, 101, 6), boats_U:(9, 100, 4), time_array:(101,)
Number of decision vars 9055
0.033448 seconds
ADDING COSTS
0.000079 secon

In [ ]:
boats = [twoInputBoat(min_interboat_clearance=0.4), threeInputBoat(min_interboat_clearance=0.4), Boat(min_interboat_clearance=0.4)]
experiments1 = run_experiments(boats, examples)

# Save Experiments

In [ ]:
label = '(init)'
with open('results/experiments'+label+'.pickle', 'wb') as f:
    pickle.dump(experiments, f)

In [22]:
label = '(min_interboat_clearance=0.4)'
with open('results/experiments'+label+'.pickle', 'wb') as f:
    pickle.dump(experiments, f)